In [38]:
import requests
import pandas as pd
from scrapy.http import HtmlResponse

In [4]:
cookies = {
    'JSESSIONID': 'F474E2915CE022928A5A77CCA69C5CC8',
}

headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://rera.karnataka.gov.in',
    'Referer': 'https://rera.karnataka.gov.in/projectViewDetails',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Not A(Brand";v="8", "Chromium";v="132", "Google Chrome";v="132"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

data = {
    'action': '12892',
}

response = requests.post(
    'https://rera.karnataka.gov.in/projectDetails', cookies=cookies, headers=headers, data=data, verify=False)
response

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/urllib3/connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rera.karnataka.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [5]:
scrapy_response = HtmlResponse(url=response.url, body=response.text, encoding="utf-8")

## Tower scraping

In [16]:
final_selector = scrapy_response.xpath('.//h1[contains(text(), "Development") and span[contains(text(), "Bifurcation")]]/..')[0]

In [22]:
towers = final_selector.xpath('./table/following-sibling::div[not(@class)]')
len(towers)

6

In [30]:
tr_eles = towers[0].xpath('./table/tr')

In [31]:
column_names = tr_eles[0].xpath('./td/text()').extract()
column_names

['Tower Name', '1', 'Type', 'Residential']

In [50]:
table_data = []
for tr_ele in tr_eles[1:]:
    if len(tr_ele.xpath('./td')) == 0 and len(tr_ele.xpath('./th')) > 0:
        print("this is header of a new table")
        break
    row = {}
    for column_name in column_names:
        row[column_name] = tr_ele.xpath(f'./td[{column_names.index(column_name) + 1}]/text()').extract_first()
    table_data.append(row)
len(table_data)

this is header of a new table


4

In [41]:
pd.DataFrame(table_data)

,Tower Name,1,Type,Residential
0,No. of Floors,32,Total No. of Units,130
1,No. of Stilts,2,No. of slab of super structure,34
2,No. of Basement,2,Total No. of Parking,130
3,Height of the Tower (In Meters),101.05,None,None


In [54]:
floor_plan = towers[0].xpath('./table/tbody[1]')[0]

In [52]:
column_names = tr_eles[-1].xpath('./th/text()').extract()
column_names

['Floor No', 'No of Units']

In [64]:
tr_eles = floor_plan.xpath('.//tr')
print(len(tr_eles))

table_data = []
for tr_ele in tr_eles[:]:
    if len(tr_ele.xpath('./td')) == 0 and len(tr_ele.xpath('./th')) > 0:
        print("this is header of a new table")
        break
    row = {}
    for column_name in column_names:
        row[column_name] = tr_ele.xpath(f'./td[{column_names.index(column_name) + 1}]/text()').extract_first()
    table_data.append(row)
len(table_data)

34
this is header of a new table


33

In [65]:
pd.DataFrame(table_data).head()

,Floor No,No of Units
0,0,3
1,1,3
2,2,4
3,3,4
4,4,4


In [66]:
floor_details = towers[0].xpath('./table/tbody[2]')[0]

In [67]:
column_names = tr_eles[-1].xpath('./th/text()').extract()
column_names

['Sl No',
 'Floor No',
 'Unit No',
 'Unit Type',
 'Carpet Area',
 'Exclusive Common Area Allottee',
 'Common Area Alloted To Association',
 'Undivided Share',
 'No of parking lots']

In [68]:
tr_eles = floor_details.xpath('.//tr')
print(len(tr_eles))

table_data = []
for tr_ele in tr_eles[:]:
    if len(tr_ele.xpath('./td')) == 0 and len(tr_ele.xpath('./th')) > 0:
        print("this is header of a new table")
        break
    row = {}
    for column_name in column_names:
        row[column_name] = tr_ele.xpath(f'./td[{column_names.index(column_name) + 1}]/text()').extract_first()
    table_data.append(row)
len(table_data)

130


130

In [70]:
pd.DataFrame(table_data).head()

,Sl No,Floor No,Unit No,Unit Type,Carpet Area,Exclusive Common Area Allottee,Common Area Alloted To Association,Undivided Share,No of parking lots
0,1,0,1001,4 BHK,149.13,8.02,43.09,53.51,1
1,2,0,1002,4 BHK,149.13,8.02,41.48,51.57,1
2,3,0,1003,3.5 BHK,128.75,8.02,34.42,43.10,1
3,4,1,1011,4 BHK,149.13,8.02,43.09,53.51,1
4,5,1,1012,4 BHK,149.13,8.02,41.48,51.57,1
